Setup Instructions
==========

These instructions install `miniconda` and the requisite libraries for the tutorial in a `conda` environment, which is an indepenent python installation.

Jupyter has the similar concept of _kernels_, which are independent execution environments. They don't even have to be Python, kernels for other languages exist as well.

By loading a separate kernel for each project, we avoid the complication of different components/projects having weird interactions, ultimately helping reproducibility.

We first produce a new conda environment with the libraries we require, then we tell Jupyter about this new environment with the ipython executable

NOTE: This step may take a little while

In [1]:
%%bash
#!/bin/bash

KERNEL_NAME="machine-learning-hats-2020"

set -e

# Download miniconda3
if [[ ! -f $HOME/miniconda3.sh ]]; then
    wget --progress=dot:giga https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O $HOME/miniconda3.sh
fi

# Install miniconda3 and activate environment
if [[ ! -d $HOME/miniconda3 ]]; then
    bash $HOME/miniconda3.sh -b -f -u -p $HOME/miniconda3
    source $HOME/miniconda3/etc/profile.d/conda.sh
    conda env create -f environment.yml --name $KERNEL_NAME
else
    source $HOME/miniconda3/etc/profile.d/conda.sh
fi
conda activate $KERNEL_NAME

# Make a wrapper script to load miniconda python
if [[ ! -f python_wrapper.sh ]]; then
cat << 'EOF' > python_wrapper.sh
#!/bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate machine-learning-hats-2020
exec python "$@"
EOF
chmod +x python_wrapper.sh
fi

# Create the kernel
if [[ ! -d $HOME/.local/share/jupyter/kernels/$KERNEL_NAME ]]; then
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF
fi

# Report OK
echo "Loaded $KERNEL_NAME!"

PREFIX=/home/cms.woodson/miniconda3
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/cms.woodson/miniconda3

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py37_0
    - cffi==1.14.0=py37he30daa8_1
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.1=py37h7b6447c_0
    - conda==4.8.3=py37_0
    - cryptography==2.9.2=py37h1ba5d50_0
    - idna==2.9=py_1
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.3=he6710b0_1
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1g=h7b6447c_0
    - pip==20.0.2=py37_3
    - pycosat==0.6.3=py37h7b6447c_0
    - pycparser==2.20=py_0
    - pyopenssl==19.1.0=py37_0
    - pysocks==1.7.1=py37_0
    - python==3.7.7=hcff3b4d_5
    - readline==8.0=h7b6447c_0
    - requests==2.23.0=py37_0
    - ruamel_yaml=

--2020-06-20 17:03:04--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘/home/cms.woodson/miniconda3.sh’

     0K ........ ........ ........ ........ 37%  105M 1s
 32768K ........ ........ ........ ........ 75%  102M 0s
 65536K ........ ........ ....             100%  120M=0.8s

2020-06-20 17:03:04 (107 MB/s) - ‘/home/cms.woodson/miniconda3.sh’ saved [88867207/88867207]



Results
=======

If successful, you should see something similar to the following:

```
Loaded machine-learning-hats-2020!
```

The new kernel you just made will then show up in the various Jupyter after refreshing the page.